# PyTorch Metric Learning on Papyri Data

## Prepare Training

### Install the necessary packages

In [ ]:
!pip install -q pytorch-metric-learning[with-hooks]
!pip install umap-learn
!pip install gpustat
!pip install efficientnet_pytorch

### Mount Google Drive with the Data. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Load the required data and files into the temporary directory.

In [ ]:
!cp gdrive/My\ Drive//mt/src/helpers.py .
!cp gdrive/My\ Drive//mt/src/models.py .
!cp gdrive/My\ Drive//mt/src/features.py .

In [ ]:
from pathlib import Path

train_data = Path("./train/")

if not train_data.is_dir():
    !unzip gdrive/My\ Drive//mt/data/train.zip
    !unzip gdrive/My\ Drive//mt/data/test.zip
    !unzip gdrive/My\ Drive//mt/data/val.zip

### Import packages

In [ ]:
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import record_keeper
import torch
import torch.nn as nn
import torchvision
import umap
import toml
from cycler import cycler
import PIL
from PIL import Image
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
import pytorch_metric_learning
import pytorch_metric_learning.utils.logging_presets as logging_presets
from pytorch_metric_learning import losses, miners, samplers, testers, trainers
from pytorch_metric_learning.utils import common_functions
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
import os
from torchsummary import summary
import helpers
import models
import features

### Initialize Notebook

In [ ]:
%matplotlib inline

### Load settings & hyperparameters from config from config file

In [ ]:
config = toml.load('./gdrive/MyDrive/mt/conf/conf.toml')
setting = config.get('settings')
param = config.get('params')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Create directory for results

In [ ]:
output_dir = helpers.create_output_dir(setting['output'], setting['experiment_name'])

### Save config

In [ ]:
!mkdir $output_dir/config/
!cp ./gdrive/MyDrive/mt/conf/conf.toml $output_dir/config/conf.toml

### Create Output Dir for UMAP Plots

In [ ]:
!mkdir ./umap_train/
!mkdir ./umap_val/

mkdir: cannot create directory ‘./umap_train/’: File exists
mkdir: cannot create directory ‘./umap_val/’: File exists


### Initialize Logger

In [ ]:
logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s" % pytorch_metric_learning.__version__)

INFO:root:VERSION 1.1.0


### Initilize Trunk Model

In [ ]:
# Set trunk model and replace the softmax layer with an identity function

if param['trunk'] == 'efficientnet_b0':
    #trunk = torchvision.models.efficientnet_b0(pretrained=False)
    trunk = EfficientNet.from_name('efficientnet-b0')    
    trunk_output_size = trunk._fc.in_features
    trunk._fc = common_functions.Identity()    
elif param['trunk'] == 'resnet50':
    trunk = torchvision.models.resnet50(pretrained=False)
    trunk_output_size = trunk.fc.in_features
    trunk.fc = common_functions.Identity()
elif param['trunk'] == 'resnet34':
    trunk = torchvision.models.resnet34(pretrained=False)
    trunk_output_size = trunk.fc.in_features
    trunk.fc = common_functions.Identity()
elif param['trunk'] == 'resnet18':
    trunk = torchvision.models.resnet18(pretrained=False)
    trunk_output_size = trunk.fc.in_features
    trunk.fc = common_functions.Identity()
elif param['trunk'] == 'densenet121':
    trunk = torchvision.models.densenet121(pretrained=False)
    trunk_output_size = trunk.classifier.in_features
    trunk.classifier = common_functions.Identity() 
trunk = trunk.to(device)

In [ ]:
trunk_output_size

1024

In [ ]:
trunk_optimizer = torch.optim.Adam(
    trunk.parameters(),
    lr=param['trunk_optimizer_lr'],
    weight_decay=param['trunk_optimizer_weight_decay'])

### Initilize Embedder Model

In [ ]:
# Set embedder model. This takes in the output of the trunk and outputs 64 dimensional embeddings
embedder = models.MLP([trunk_output_size, 64]).to(device)

In [ ]:
embedder_optimizer = torch.optim.Adam(
    embedder.parameters(),
    lr=param['embedder_optimizer_lr'],
    weight_decay=param['embedder_optimizer_weight_decay']
)

### Initilize Classifier Model

In [ ]:
# Set the classifier. The classifier will take the embeddings and output a 50 dimensional vector.
# (Our training set will consist of the first 50 classes of the CIFAR100 dataset.)
# We'll specify the classification loss further down in the code.
classifier = models.MLP([64, 50]).to(device)

In [ ]:
classifier_optimizer = torch.optim.Adam(
    classifier.parameters(),
    lr=param['classifier_optimizer_lr'],
    weight_decay=param['classifier_optimizer_weight_decay'])

In [ ]:
# In the final case and if we want to have a model summary
if False:
    %%capture cap --no-stderr
    with open(output_dir + '/config/trunk_summary.txt', 'w') as f:
        f.write(cap.stdout)
        print(summary(trunk, (3, 64,64)))

In [ ]:
# In the final case and if we want to have a model summary
if False:
    %%capture cap --no-stderr
    with open(output_dir + '/config/embedder_summary.txt', 'w') as f:
        f.write(cap.stdout)
        print(summary(embedder, (1,64)))

In [ ]:
# In the final case and if we want to have a model summary
if False:
    %%capture cap --no-stderr
    with open(output_dir + '/config/classifier_summary.txt', 'w') as f:
        f.write(cap.stdout)
        print(summary(embedder, (1,64)))

### Initialize Data Transformations

In [ ]:
train_transform = transforms.Compose([    
    transforms.ColorJitter(),                                      
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((param['normalize_1'], param['normalize_2'], param['normalize_3']),
                        (param['normalize_4'], param['normalize_5'], param['normalize_6']))]
)

val_transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((param['normalize_1'], param['normalize_2'], param['normalize_3']),
                        (param['normalize_4'], param['normalize_5'], param['normalize_6']))]
)

### Initilize Datasets

In [ ]:
train_dataset = features.PypyrusDataset(data=setting['path_train'],
                               csv=setting['csv'],
                               mode='train',
                               transform=train_transform,
                               debug=False,
                               batch_size=64)

val_dataset = features.PypyrusDataset(data=setting['path_val'],
                               csv=setting['csv'],
                               mode='val',
                               transform=train_transform,
                               debug=False,
                               batch_size=64)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


### Check if datasats are class disjoint

In [ ]:
assert set(train_dataset.targets).isdisjoint(set(val_dataset.targets))

### Initialize Loss Function


In [ ]:
# Set the loss function
if param['metric_loss'] == 'TripletMarginLoss':
    loss = losses.TripletMarginLoss(margin=0.1)
elif param['metric_loss'] == 'CentroidTripletLoss':
    loss = losses.CentroidTripletLoss(margin=0.1)
elif param['metric_loss'] == 'AngularLoss':
    loss = losses.AngularLoss(alpha=40)
elif param['metric_loss'] == 'MultiSimilarityLoss':
    loss = losses.MultiSimilarityLoss(alpha=2, beta=50, base=0.5)

# Set the classification loss:
classification_loss = torch.nn.CrossEntropyLoss()

### Initialize Miner & Sampler

In [ ]:
# Set the mining function
if param['miner'] == 'MultiSimilarityMiner':
    miner = miners.MultiSimilarityMiner(epsilon=0.1)
elif param['miner'] == 'TripletMarginMiner':
    miner = miners.TripletMarginMiner(margin=0.2, type_of_triplets="all")
elif param['miner'] == 'AngularMiner':
    miner = miners.AngularMiner(angle=20)
# Set the dataloader sampler
sampler = samplers.MPerClassSampler(train_dataset.targets, m=4, length_before_new_iter=len(train_dataset))

### Set Training Hyperparameters 

In [ ]:
batch_size = param['batch_size']
num_epochs = param['num_epochs']

### Prepare Dictionaries for Deep Metric Learning Trainer

In [ ]:
# Package the above stuff into dictionaries.
models = {"trunk": trunk, "embedder": embedder, "classifier": classifier}
optimizers = {
    "trunk_optimizer": trunk_optimizer,
    "embedder_optimizer": embedder_optimizer,
    "classifier_optimizer": classifier_optimizer,
}
loss_funcs = {"metric_loss": loss, "classifier_loss": classification_loss}
mining_funcs = {"tuple_miner": miner}

### Specify loss weights

In [ ]:
loss_weights = {"metric_loss": 1, "classifier_loss": 0.5}

### Get records for the tensorboard

In [ ]:
record_keeper, _, _ = logging_presets.get_record_keeper(
    output_dir + "/logs", output_dir + "/tensorboard"
)
hooks = logging_presets.get_hook_container(record_keeper)
dataset_dict = {"train":train_dataset,"val": val_dataset}
model_folder = output_dir + "/saved_models"

### Create Deep Metric Tester

In [ ]:
tester = testers.GlobalEmbeddingSpaceTester(
    end_of_testing_hook=hooks.end_of_testing_hook,
    visualizer=umap.UMAP(),
    visualizer_hook=helpers.visualizer_hook,
    dataloader_num_workers=2,
    accuracy_calculator=AccuracyCalculator(k="max_bin_count"),
    set_min_label_to_zero=True,
    dataset_labels = list(np.unique(train_dataset.targets)) + list(np.unique(val_dataset.targets)),
)

### Initialie Tester as Hook at End of Epoch

In [ ]:
end_of_epoch_hook = hooks.end_of_epoch_hook(
    tester, dataset_dict, model_folder, test_interval=param['test_interval'], patience=param['patience']
)

### Initialize Scheduler 


In [ ]:
lr_schedulers_dict = {"trunk_scheduler_by_plateau": torch.optim.lr_scheduler.ReduceLROnPlateau(trunk_optimizer),
                      "embedder_scheduler_by_plateau": torch.optim.lr_scheduler.ReduceLROnPlateau(embedder_optimizer),
                      "classifier_scheduler_by_plateau": torch.optim.lr_scheduler.ReduceLROnPlateau(classifier_optimizer),
                      }

### Initialize Final Deep Metric Learning Trainer Function

In [ ]:
trainer = trainers.TrainWithClassifier(
    models,
    optimizers,
    batch_size,
    loss_funcs,    
    mining_funcs,
    train_dataset,
    set_min_label_to_zero = True,
    lr_schedulers = lr_schedulers_dict,
    dataset_labels = list(np.unique(train_dataset.targets)),
    dataloader_num_workers=2,
    loss_weights=loss_weights,
    end_of_iteration_hook=hooks.end_of_iteration_hook,
    end_of_epoch_hook=end_of_epoch_hook,
)

## Start Training

In [ ]:
trainer.train(num_epochs=num_epochs)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!cp -R ./umap_train/ $output_dir
!cp -R ./umap_val/ $output_dir